LLMを使用して python のコーディングエラーの修正案を提示する内容です
参考内容は java のコードを解析するために LSPサーバーを起動してファイル解析やコード保管などの文法間違いの判定を行なっていますが python での実行野ために修正案コードに対して簡単な後処理を実行してみます

- Refarence:
    - [Copiloting the Copilots: Fusing Large Language Models with Completion Engines for Automated Program Repair](https://arxiv.org/abs/2309.00608v3)
    - [Repilot](https://github.com/ise-uiuc/Repilot)

In [1]:
!python3 -m pip install --upgrade pip

In [2]:
# !pip install openai==1.2.3
!pip install openai==1.3.4

In [3]:
!pip install python-dotenv tiktoken

In [4]:
!pip install pdfplumber

In [6]:
# エラー用のコード
# 三目並べ 
# 行 列 と入力したときにその位置に手を指す
import random
from IPython.display import clear_output

def print_board(board):
    # clear_output(wait=True)
    for row in board:
        print(" | ".join(row))
        print("-" * 5)

def check_winner(board, player):
    for row in board:
        if all(s == player for s in row):
            return True
    for col in range(3):
        if all(board[row][col] == player for row in range(3)):
            return True
    if all(board[i][i] == player for i in range(3)):
        return True
    if all(board[i][2 - i] == player for i in range(3)):
        return True
    return False

def get_empty_cells(board):
    return [(i, j) for i in range(3) for j in range(3) if board[i][j] == " "]


def find_block_move(board, player):
    for row in range(3):
        if board[row].count(player) == 2 and " " in board[row]:
            return row, board[row].index(" ")

    for col in range(3):
        column = [board[row][col] for row in range(3)]
        if column.count(player) == 2 and " " in column:
            return column.index(" "), col

    diagonal1 = [board[i][i] for i in range(3)]
    if diagonal1.count(player) == 2 and " " in diagonal1:
        return diagonal1.index(" "), diagonal1.index(" ")

    diagonal2 = [board[i][2 - i] for i in range(3)]
    if diagonal2.count(player) == 2 and " " in diagonal2:
        return diagonal2.index(" "), 2 - diagonal2.index(" ")

    return None

def auto_place_o(board):
    block_move = find_block_move(board, "X")
    if block_move:
        row, col = block_move
        board[row][col] = "O"
    else:
        empty_cells = get_empty_cells(board)
        if empty_cells:
            row, col = random.choice(empty_cells)
            board[row][col] = "O"

def tic_tac_toe_jupyter():
    board = [[" " for _ in range(3)] for _ in range(3)]
    current_player = "X"
    while True:
        print_board(board)
        if current_player == "X":
            try:
                row, col = map(int, input("Player X, enter row and column (1-3): ").split())
                row, col = row - 1, col - 1
                if board[row][col] != " ":
                    print("Cell is already taken. Please try again.")
                    continue
            except (ValueError, IndexError):
                print("Invalid input. Please enter row and column numbers from 1 to 3.")
                continue
            board[row][col] = "X"
        else:
            auto_place_o(board)

        if check_winner(board, current_player):
            print_board(board)
            print(f"Player {current_player} wins!")
            break

        if all(all(cell != " " for cell in row) for row in board):
            print_board(board)
            print("It's a tie!")
            break

        current_player = "O" if current_player == "X" else "X"

    if input("Play again? (yes/no): ").lower().startswith("y"):
        tic_tac_toe_jupyter()

tic_tac_toe_jupyter()

  |   |  
-----
  |   |  
-----
  |   |  
-----


Player X, enter row and column (1-3):  2 2


  |   |  
-----
  | X |  
-----
  |   |  
-----
  |   |  
-----
  | X | O
-----
  |   |  
-----


Player X, enter row and column (1-3):  1 1


X |   |  
-----
  | X | O
-----
  |   |  
-----
X |   |  
-----
  | X | O
-----
  |   | O
-----


Player X, enter row and column (1-3):  1 3


X |   | X
-----
  | X | O
-----
  |   | O
-----
X | O | X
-----
  | X | O
-----
  |   | O
-----


Player X, enter row and column (1-3):  3 1


X | O | X
-----
  | X | O
-----
X |   | O
-----
Player X wins!


Play again? (yes/no):  no


In [7]:
from contextlib import contextmanager
from time import time

class Timer:
    """処理時間を表示するクラス
    with Timer(prefix=f'pred cv={i}'):
        y_pred_i = predict(model, loader=test_loader)
    
    with Timer(prefix='fit fold={} '.format(i)):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)

    with Timer(prefix='fit fold={} '.format(i), verbose=500):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)
    """
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' ', verbose=0):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None
        self.verbose = verbose

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [8]:
import openai
import pdfplumber
from openai import OpenAI
import tiktoken
from dotenv import load_dotenv
import os
load_dotenv()

True

In [9]:
# OpenAIのLLMを使って初期提案を生成する関数
def generate_initial_proposal_with_openai(buggy_code, error_log):
    openai.api_key = os.getenv("OPENAI_API_KEY")
    client = OpenAI()  # OpenAIクライアントの初期化
    response = client.chat.completions.create(
        # model="gpt-4-1106-preview",  # モデルの指定
        model="gpt-3.5-turbo-1106",  # モデルの指定
        messages=[
            {"role": "system", "content": "You are a programmer who takes an error log and corresponding code as input and generates code to correct the error."},
            {"role": "user", "content": f"以下のコードにはエラーがあります：\n{buggy_code}\nエラーログ：\n{error_log}\nこのエラーを修正するためのコードを提案してください。\n"}
        ],
        temperature=1.8
    )
    return response.choices[0].message.content

In [12]:
# 元ネタの https://github.com/ise-uiuc/Repilot では
# java の文法チェック、コードの整合性の確認を LSP サーバーを起動して確認するが
# それの python 版

# class JdtLspAnalyzer:
#     # JdtLspAnalyzer クラスの概念的な実装
#     def __init__(self):
#         pass

#     def analyze(self, code):
#         # Javaコードの解析を行う概念的なメソッド
#         return {"error_info": "ダミーの解析結果"}

# def repair_function(proposal, analysis_result):
#     # 修復提案を改善する概念的な関数
#     return proposal + "\n// 修正提案が改善されました"

# def improve_proposal_with_repilot(initial_proposal, buggy_code):
#     # JdtLspAnalyzerを使用してJavaコードを解析
#     conn = Client(...)  # 適切な接続情報を設定
#     analyzer = JdtLspAnalyzer(conn, server_cmd, proj_path, ...)
#     analyzer.start()
#     analysis_result = analyzer.analyze(buggy_code)  # Javaコードを解析
#     analyzer.terminate()
    
#     # 修正提案を改善
#     improved_proposal = initial_proposal  # ここに修正提案の改善処理を追加
#     return improved_proposal

import ast

def improve_proposal_with_repilot(initial_proposal, buggy_code):
    improved_proposal = initial_proposal

    # 文法チェック - 提案されたコードが有効なPythonコードであるか確認
    try:
        ast.parse(improved_proposal)
        # ここで追加の改善処理を実行できます（例：コードの再フォーマット）
    except SyntaxError:
        # 文法エラーがある場合、改善された提案をエラーメッセージに置き換える
        improved_proposal = "# 文法エラーが検出されました。\n" + initial_proposal

    return improved_proposal

In [11]:
# エラーになる処理の追加
def tic_tac_toe_jupyter():
    board = [[" " for _ in range(3)] for _ in range(3)]
    current_player = "X"
    turns = 0  # ターン数を追跡

    while True:
        print_board(board)
        if current_player == "X":
            try:
                row, col = map(int, input("Player X, enter row and column (1-3): ").split())
                row, col = row - 1, col - 1
                if board[row][col] != " ":
                    print("Cell is already taken. Please try again.")
                    continue
            except (ValueError, IndexError):
                print("Invalid input. Please enter row and column numbers from 1 to 3.")
                continue
            board[row][col] = "X"
        else:
            auto_place_o(board)

        turns += 1
        if turns == 3:  # エラーを導入
            current_player = current_player[1]  # 不正なインデックスアクセス

        if check_winner(board, current_player):
            print_board(board)
            print(f"Player {current_player} wins!")
            break

        if all(all(cell != " " for cell in row) for row in board):
            print_board(board)
            print("It's a tie!")
            break

        current_player = "O" if current_player == "X" else "X"

    if input("Play again? (yes/no): ").lower().startswith("y"):
        tic_tac_toe_jupyter()
tic_tac_toe_jupyter()

  |   |  
-----
  |   |  
-----
  |   |  
-----


Player X, enter row and column (1-3):  2 2


  |   |  
-----
  | X |  
-----
  |   |  
-----
  |   |  
-----
  | X |  
-----
O |   |  
-----


Player X, enter row and column (1-3):  1 1


IndexError: string index out of range

In [13]:
buggy_code = "ここにエラーのあるコードを入力"
buggy_code = """
def tic_tac_toe_jupyter():
    board = [[" " for _ in range(3)] for _ in range(3)]
    current_player = "X"
    turns = 0  # ターン数を追跡

    while True:
        print_board(board)
        if current_player == "X":
            try:
                row, col = map(int, input("Player X, enter row and column (1-3): ").split())
                row, col = row - 1, col - 1
                if board[row][col] != " ":
                    print("Cell is already taken. Please try again.")
                    continue
            except (ValueError, IndexError):
                print("Invalid input. Please enter row and column numbers from 1 to 3.")
                continue
            board[row][col] = "X"
        else:
            auto_place_o(board)

        turns += 1
        if turns == 3:  # エラーを導入
            current_player = current_player[1]  # 不正なインデックスアクセス

        if check_winner(board, current_player):
            print_board(board)
            print(f"Player {current_player} wins!")
            break

        if all(all(cell != " " for cell in row) for row in board):
            print_board(board)
            print("It's a tie!")
            break

        current_player = "O" if current_player == "X" else "X"

    if input("Play again? (yes/no): ").lower().startswith("y"):
        tic_tac_toe_jupyter()
"""
error_log = "ここにエラーログを入力"
error_log = """
---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
Cell In[11], line 40
     38     if input("Play again? (yes/no): ").lower().startswith("y"):
     39         tic_tac_toe_jupyter()
---> 40 tic_tac_toe_jupyter()

Cell In[11], line 24, in tic_tac_toe_jupyter()
     22 turns += 1
     23 if turns == 3:  # エラーを導入
---> 24     current_player = current_player[1]  # 不正なインデックスアクセス
     26 if check_winner(board, current_player):
     27     print_board(board)

IndexError: string index out of range
"""
initial_proposal = generate_initial_proposal_with_openai(buggy_code, error_log)
improved_proposal = improve_proposal_with_repilot(initial_proposal, buggy_code)

print("改善された修正提案:", improved_proposal)

改善された修正提案: # 文法エラーが検出されました。
エラーの修正には、`current_player` の更新方法を修正する必要があります。現在のコードでは、`current_player` を文字列として扱っていますが、1文字目のみを取得しようとしているため、エラーが発生しています。

以下の修正を提案します：
```python
if turns == 3:
    current_player = "O" if current_player == "X" else "X"
```

修正後のコードは以下の通りです：

```python
def tic_tac_toe_jupyter():
    board = [[" " for _ in range(3)] for _ in range(3)]
    current_player = "X"
    turns = 0  # ターン数を追跡

    while True:
        print_board(board)
        if current_player == "X":
            try:
                row, col = map(int, input("Player X, enter row and column (1-3): ").split())
                row, col = row - 1, col - 1
                if board[row][col] != " ":
                    print("Cell is already taken. Please try again.")
                    continue
            except (ValueError, IndexError):
                print("Invalid input. Please enter row and column numbers from 1 to 3.")
                continue
            board[row][col] = "X"
        else:
         

In [14]:
# 提案されたコードを試す
def tic_tac_toe_jupyter():
    board = [[" " for _ in range(3)] for _ in range(3)]
    current_player = "X"
    turns = 0  # ターン数を追跡

    while True:
        print_board(board)
        if current_player == "X":
            try:
                row, col = map(int, input("Player X, enter row and column (1-3): ").split())
                row, col = row - 1, col - 1
                if board[row][col] != " ":
                    print("Cell is already taken. Please try again.")
                    continue
            except (ValueError, IndexError):
                print("Invalid input. Please enter row and column numbers from 1 to 3.")
                continue
            board[row][col] = "X"
        else:
            auto_place_o(board)

        turns += 1
        if turns == 3:
            current_player = "O" if current_player == "X" else "X"

        if check_winner(board, current_player):
            print_board(board)
            print(f"Player {current_player} wins!")
            break

        if all(all(cell != " " for cell in row) for row in board):
            print_board(board)
            print("It's a tie!")
            break

        current_player = "O" if current_player == "X" else "X"

    if input("Play again? (yes/no): ").lower().startswith("y"):
        tic_tac_toe_jupyter()
        
tic_tac_toe_jupyter()

  |   |  
-----
  |   |  
-----
  |   |  
-----


Player X, enter row and column (1-3):  2 2


  |   |  
-----
  | X |  
-----
  |   |  
-----
  |   |  
-----
  | X | O
-----
  |   |  
-----


Player X, enter row and column (1-3):  1 2


  | X |  
-----
  | X | O
-----
  |   |  
-----


Player X, enter row and column (1-3):  3 2


  | X |  
-----
  | X | O
-----
  | X |  
-----
Player X wins!


Play again? (yes/no):  no


動くようにはなったけどゲームとして成り立たないという・・・